# Notebook d'entraînement 
L'idée de ce notebook est de faciliter le lancement et la récupération des modèles entraînés en capitalisant sur les codes existants.

In [1]:
import torch.nn as nn
import sklearn
import torch
import pickle
import os
import time
from tqdm import tqdm
from joblib import Parallel, delayed
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda")

In [2]:
#!pip install unidecode
#!pip install transformers
#!pip install networkx
from fats import F1_score,Weighted_Loss,Simple_Classifier,Multi_Linear_Classifier,SMHA_Linear_classifier,Net,training_loop_gpu

In [10]:
dataloader_2=pickle.load(open('train_loader_2.pickle','rb'))
print("Nombre de batchs :",len(dataloader_2))
for _,batch in enumerate(dataloader_2):
    print("Taille du batch :",len(batch[0]))
    break

Nombre de batchs : 2095
Taille du batch : 64


In [3]:
#!pip install transformers
from transformers import CamembertModel,CamembertConfig,AdamW
camem1=CamembertModel(CamembertConfig())
camem2=CamembertModel.from_pretrained("camembert-base")

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Prépration des modèles et fonctions de pertes

Ici on ne va déclarer que les modèles et pas les optimiseurs car il faut d'abord placer le modèle sur GPU puis ensuite déclarer l'optimiseur, sinon ce dernier peut se tromper entre les devices (entre GPU et CPU donc).

In [8]:
mlc=Multi_Linear_Classifier(camem2.config.hidden_size)
#mlc_optimizer_SGD=optim.SGD(mlc.parameters(), lr=0.001, momentum=0.09)
#mlc_optimizer_Adam=optim.AdamW(mlc.parameters(), lr=0.009, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)

slc=Simple_Classifier(camem2.config.hidden_size)
#slc_optimizer=optim.AdamW(slc.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
#SGD(slc.parameters(), lr=0.001, momentum=0.09)

att_lin_c=SMHA_Linear_classifier(torch.Size([512,768]),8,768)
#path='SMHA_Linear_classifier.pt'
#att_lin_c.load_state_dict(torch.load(path))
#att_lin_c_optimizer=optim.AdamW(att_lin_c.parameters(), lr=0.009, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
#.SGD(att_lin_c.parameters(), lr=0.001, momentum=0.09)
#att_lin_c_optimizer.load_state_dict(torch.load(path[:-3]+'_AdamW.pt'))

convnet=Net(2**8,2**6,2,2,2,2)
#convnet_optimizer=optim.AdamW(convnet.parameters(), lr=0.009, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
#SGD(convnet.parameters(), lr=0.001, momentum=0.09)

In [11]:
score=F1_score()

alpha=0.1
weights=torch.Tensor([(1/(1-alpha))*1/((512-20)/512),(1/alpha)*1/((20)/512)])
#weights=torch.Tensor([(1/(1-alpha))*1/((1)/512),(1/alpha)*1/((512-1)/512)])
#weights=torch.Tensor([1,1])
print(weights)
loss_3=Weighted_Loss(weight=weights,loss_type='sum',binary=False)
loss_2=Weighted_Loss(weight=weights,loss_type='L1',binary=False)
loss=nn.MSELoss()#nn.L1Loss()
Loss=[loss,loss_2,loss_3]
epochs=10

Models=[mlc,slc,att_lin_c,convnet]

data=dataloader_2

tensor([  1.1563, 256.0000])


In [12]:
import gc
def check_weights_update(model,loss,optim,dev=False):
    param1=list(model.parameters())[0].clone()
    input=torch.rand(torch.Size([3,512,768]))
    target=torch.rand(torch.Size([3,512]))
    
    if dev:
        input=input.to(torch.device('cuda'))
        target=target.to(torch.device('cuda'))
    
    sortie=model(input)
    optim.zero_grad()
    ouais=loss(sortie,target)
    ouais.backward()
    optim.step()
    param=list(model.parameters())[0].clone()
    out=bool(1-torch.equal(param1.data,param.data))
    del input,target,sortie,ouais
    gc.collect()
    torch.cuda.empty_cache()
    return out
#check_weights_update(Models[0],Loss[0],Optimizers[0])

## Première boucle d'entraînement
D'abord un premier entraînement, on récupère les modèles

In [ ]:
for num_model in range(len(Models)):
    
    Models[num_model]=Models[num_model].to(device)
    optimizer=optim.AdamW(Models[num_model].parameters(), lr=0.009, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
    
    if check_weights_update(Models[num_model],Loss[0],optimizer,dev=True):
        
        model,optimizer,trainin_stats=training_loop_gpu( #On entraîne les modèles au fur et à mesure
            Models[num_model],
            optimizer,
            data,
            score,
            Loss,
            epochs,
            camem2,
            device
        )
        
        pickle.dump(training_stats,open('training_stats_'+str(Models[num_model]).split('(')[0],'wb'))
        torch.save(model.state_dict(), str(Models[num_model]).split('(')[0]+".pt")
        torch.save(optimizer.state_dict(), str(Models[num_model]).split('(')[0]+'_'+str(optimizer).split(' (')[0]+".pt")
        print("Model saved!")        
        Models[num_model]=Models[num_model].to('cpu')
    
    else:
        print("Le modèle",str(Models[num_model]).split('(')[0],"n'a pas pu être entraîné !")
    

  0%|          | 0/2095 [00:00<?, ?it/s]

Entraînement du modèle : Multi_Linear_Classifier


 69%|██████▉   | 1453/2095 [7:43:25<3:41:58, 20.75s/it]

## Deuxième entraînement
On reprend les modèles et on poursuit l'entraînement

In [13]:
Models_2=pickle.load(open('Models.pickle','rb'))
Models_2[0]

Multi_Linear_Classifier(
  (linear1): Linear(in_features=768, out_features=384, bias=True)
  (linear2): Linear(in_features=384, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=1, bias=True)
  (Lrelu): LeakyReLU(negative_slope=0.01)
  (softmax): Softmax(dim=-1)
)

In [15]:
Models[0].load_state_dict(Models_2[0].state_dict())


<All keys matched successfully>

In [ ]:
Optimizer=[]

for num_model in range(len(Models)):
    
    Models[num_model].load_state_dict(Models_2[num_model].state_dict())
    Models[num_model]=Models[num_model].to(device)
    optimizer=optim.AdamW(Models[num_model].parameters(), lr=0.009, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
    
    if check_weights_update(Models[num_model],Loss[0],optimizer,dev=True):
        
        model,optimizer,trainin_stats=training_loop_gpu( #On entraîne les modèles au fur et à mesure
            Models[num_model],
            optimizer,
            data,
            score,
            Loss,
            epochs,
            camem2,
            device
        )
        
        Optimizer.append(optimizer)
        pickle.dump(training_stats,open('training_stats_'+str(Models[num_model]).split('(')[0],'wb'))
        torch.save(model.state_dict(), str(Models[num_model]).split('(')[0]+".pt")
        torch.save(optimizer.state_dict(), str(Models[num_model]).split('(')[0]+'_'+str(optimizer).split(' (')[0]+".pt")
        print("Model saved!")        
        Models[num_model]=Models[num_model].to('cpu')
    
    else:
        print("Le modèle",str(Models[num_model]).split('(')[0],"n'a pas pu être entraîné !")
    

  0%|          | 0/2095 [00:00<?, ?it/s]

Entraînement du modèle : Multi_Linear_Classifier


 28%|██▊       | 595/2095 [3:47:22<9:39:58, 23.20s/it] 

In [18]:
pickle.dump(Models,open('Models_v2.pickle','wb'))
pickle.dump(Optimizer,open('Optimizer_v2.pickle','wb'))

In [42]:
pickle.dump(trainin_stats,open('training_stat_mystere.pickle','wb'))

In [ ]:
tv=camem2(ouais[0],ouais[1])
y=Models[2](tv.last_hidden_state.to(device))
score(y,ouais[4].to(device))

In [23]:
Models[0]()

Multi_Linear_Classifier(
  (linear1): Linear(in_features=768, out_features=384, bias=True)
  (linear2): Linear(in_features=384, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=1, bias=True)
  (Lrelu): LeakyReLU(negative_slope=0.01)
  (softmax): Softmax(dim=-1)
)

In [22]:
trainin_stats

[{'epoch': 1,
  'Training Loss MSE': 6.215948306254961e-05,
  'Training Loss L1': 0.02474360048930941,
  'Training Loss sum': 108.61064178539631,
  'Training f1 score': 0.04962713702414146,
  'Training precision score': 0.05459411998883974},
 {'epoch': 2,
  'Training Loss MSE': 4.2184651274488835e-05,
  'Training Loss L1': 0.02296307543314869,
  'Training Loss sum': 100.80674801555624,
  'Training f1 score': 0.057260481533116546,
  'Training precision score': 0.06058804635536415},
 {'epoch': 3,
  'Training Loss MSE': 4.08952362934527e-05,
  'Training Loss L1': 0.022724188774432944,
  'Training Loss sum': 99.75225820609664,
  'Training f1 score': 0.060648992218450604,
  'Training precision score': 0.06570418561610943},
 {'epoch': 4,
  'Training Loss MSE': 4.012278862065399e-05,
  'Training Loss L1': 0.022422738146561427,
  'Training Loss sum': 98.4216074613512,
  'Training f1 score': 0.056994129784920904,
  'Training precision score': 0.0649281363027406},
 {'epoch': 5,
  'Training Loss 

In [9]:
model,optimizer,trainin_stats=training_loop( #On entraîne les modèles au fur et à mesure
    Models[0],
    Optimizers[0],
    data,
    score,
    Loss,
    epochs,
    camem2,
    device
)

  0%|          | 0/2095 [00:00<?, ?it/s]

Entraînement du modèle : Multi_Linear_Classifier


  0%|          | 0/2095 [00:35<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [26]:
model=Models[3]
optimizer=Optimizers[3]

In [ ]:
training_stats = []
#score_stat=[]
# Boucle d'entrainement
model.train()
model.to(device)
model.zero_grad()
print("Entraînement du modèle :",str(model).split('(')[0])
len_loss=len(Loss)
start=time.time()

for epoch in range(0, epochs):

    # On initialise la loss pour cette epoque
    total_train_loss = 0
    if len_loss>1:
        total_train_loss_2 = 0
        if len_loss>2:
            total_train_loss_3 = 0
    f1_score=0
    prec_score=0

    # On met le modele en mode 'training'
    # Dans ce mode certaines couches du modele agissent differement

    # Pour chaque batch
    for step, batch in enumerate(tqdm(data)):

        # On recupere les donnees du batch
        input_id = batch[0]#.to(device)
        mask = batch[1]#.to(device)
        #clss = batch[2].float().to(device)
        #mask_cls=batch[3]#.to(device)
        output=batch[4].float().to(device)

        param1=list(model.parameters())[0].clone()

        # On met le gradient a 0
        optimizer.zero_grad()#summa_parallel.zero_grad()        

        # On passe la donnee au model et on recupere la loss et le logits (sortie avant fonction d'activation)
        topvec=camem2(input_id,mask)
        topvec=topvec.last_hidden_state.to(device)
        #topvec=topvec.mul(mask_cls.unsqueeze(2)).to(device)

        sortie=model(topvec)

        #On calcule et garde le score pour information, mais le détache pour éviter de faire exploser la mémoire
        f1_score+=score(sortie,output).detach().item()
        prec_score+=score.precision(sortie,output).detach().item()

        #output2=make_output_topk(output,k=1).long().to(device)
        loss_train=Loss[0](sortie,output)#.detach().item() # on commente detach sur la loss par rapport à laquelle on veut optimiser
        if len_loss>1:
            loss_train_2=Loss[1](sortie,output).detach().item()
            if len_loss>2:
                loss_train_3=Loss[2](sortie,output).detach().item()

        # Backpropagtion
        loss_train.backward()
        # On actualise les paramètres grace a l'optimizer
        optimizer.step()

        # Checks if the weights did update, if not, informs at which step
        param2=list(model.parameters())[0].clone()
        check=bool(1-torch.equal(param1.data,param2.data))
        if check==False:
            print("The weights did not update at batch",step,"epoch",epoch)        

        # Keep all the predictions
        #pred.append(sortie.detach())

        # .item() donne la valeur numerique de la loss
        total_train_loss += loss_train.detach().item() 
        if len_loss>1:
            total_train_loss_2 += loss_train_2#.detach().item() 
            if len_loss>2:
                total_train_loss_3 += loss_train_3#.detach().item() 

    # On calcule les statistiques et les pertes moyennes sur toute l'epoque
    f1_stat=f1_score/len(data)
    prec_stat=prec_score/len(data)
    avg_train_loss = total_train_loss / len(data)
    if len_loss>1:
        avg_train_loss_2 = total_train_loss_2 / len(data)   
        if len_loss>2:
            avg_train_loss_3 = total_train_loss_3 / len(data)   

            print("\nAverage training loss MSE: {0:.4f}".format(avg_train_loss),
                  "\nAverage training loss L1: {0:.4f}".format(avg_train_loss_2),
                  "\nAverage training loss sum: {0:.4f}".format(avg_train_loss_3),
                  "\nAverage f1 score: {0:.4f}".format(f1_stat),
                  "\nAverage precision score: {0:.4f}".format(prec_stat))  

            # Enregistrement des stats de l'epoque
            training_stats.append(
                {'epoch': epoch + 1,
                'Training Loss MSE': avg_train_loss,
                'Training Loss L1': avg_train_loss_2,
                'Training Loss sum': avg_train_loss_3,
                'Training f1 score': f1_stat,
                'Training precision score':prec_stat})
        else:

            print("\nAverage training loss MSE: {0:.4f}".format(avg_train_loss),
                  "\nAverage training loss L1: {0:.4f}".format(avg_train_loss_2),
                  "\nAverage f1 score: {0:.4f}".format(f1_stat),
                  "\nAverage precision score: {0:.4f}".format(prec_stat))  

            # Enregistrement des stats de l'epoque
            training_stats.append(
                {'epoch': epoch + 1,
                'Training Loss MSE': avg_train_loss,
                'Training Loss L1': avg_train_loss_2,
                'Training f1 score': f1_stat,
                'Training precision score':prec_stat})
    else:

            print("\nAverage training loss MSE: {0:.4f}".format(avg_train_loss),
                  "\nAverage f1 score: {0:.4f}".format(f1_stat),
                  "\nAverage precision score: {0:.4f}".format(prec_stat))  

            # Enregistrement des stats de l'epoque
            training_stats.append(
                {'epoch': epoch + 1,
                'Training Loss MSE': avg_train_loss,
                'Training f1 score': f1_stat,
                'Training precision score':prec_stat})

end=time.time()
print("L'entraînement a duré :",round((end-start)/60,2),"minutes.")
model.to('cpu')

In [21]:
Models

[Multi_Linear_Classifier(
   (linear1): Linear(in_features=768, out_features=384, bias=True)
   (linear2): Linear(in_features=384, out_features=128, bias=True)
   (linear3): Linear(in_features=128, out_features=1, bias=True)
   (Lrelu): LeakyReLU(negative_slope=0.01)
   (softmax): Softmax(dim=-1)
 ),
 Simple_Classifier(
   (linear1): Linear(in_features=768, out_features=1, bias=True)
   (relu): LeakyReLU(negative_slope=0.01)
 ),
 SMHA_Linear_classifier(
   (MHA): MultiheadAttention(
     (out_proj): _LinearWithBias(in_features=768, out_features=768, bias=True)
   )
   (LReLu): LeakyReLU(negative_slope=0.01)
   (sigmoid): Sigmoid()
   (LN): LayerNorm((512, 768), eps=1e-05, elementwise_affine=True)
   (linear1): Linear(in_features=768, out_features=384, bias=True)
   (linear2): Linear(in_features=384, out_features=128, bias=True)
 ),
 Net(
   (conv1): Conv1d(512, 512, kernel_size=(256,), stride=(2,))
   (pool): MaxPool1d(kernel_size=64, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [ ]:
!pip install gensim==3.8.3
!pip install unidecode
!pip install torch
!pip install sentencepiece
!pip install transformers
!pip install bs4
!pip install networkx

In [6]:
import fats
Models=pickle.load(open('Models_v2.pickle','rb'))

Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
for m in Models:
    name=str(m).split('(')[0]
    torch.save(m.state_dict(),name+'.pt')

In [23]:
Optimizer=pickle.load(open('Optimizer_v2.pickle','rb'))
for o in Optimizer:
    name=str(o).split('(')[0][:-1]
    torch.save(o.state_dict(),name+'.pt')

In [20]:
str(Optimizer[1]).split('(')[0]

IndexError: list index out of range

In [21]:
Optimizer

[AdamW (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 0.009
     weight_decay: 0.01
 )]